**Pipeline explanation**

This notebook is structured as follows. First, in `Section 1.`,

1. Imports

we define some auxiliary functions, to compute SDPH, to plot SDPH diagrams

Then, in each of `Sections 2., 3., 4., 5.`, we show how to generate shapes and compute their diagrams for:

2. Gaussian Random Fields
3. Curvatubes porous shapes
4. Bone marrow vessels
5. Curvatubes shapes sharing same or nearly same texture

Finally, in `Section 6.`, we show how to plot and visualize the diagrams next to the shapes.


For any query or bug, please contact me at 
` [first name].[last name].maths@gmail.com `


# Imports

In [ ]:
# Do you want to re-generate all shapes by yourself? 
# If yes: this will erase existing files. Set this variable to True. 
# Otherwise: leave as False.

ERASE_AND_SAVE_YOURS = False

In [ ]:
# set your directory path appropriately, in which we can find this file and the other folders

directory = 'user/........../SDPH_git/'

import os
import sys

print(os.getcwd())
os.chdir(directory)
print(os.getcwd())


In [ ]:
import re
import time
import pickle

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from itertools import product
from IPython.display import clear_output

import imageio


def slices(u, figsize = (12,4), rescale = True, cmap = 'gray') :
    '''Visualize three 2D slices of a 3D volume at z = Z//3, Z//2, or 2*Z//3
    rescale = True: grays between u.min and u.max
    rescale = False: grays between 0 and 1 (black/white if beyond 0/1)'''
    vmin = None ; vmax = None
    Z = u.shape[0]
    if not rescale : vmin = 0. ; vmax = 1. 
    fig, ax = plt.subplots(1, 3, figsize = figsize, sharex=True, sharey=True)
    ax[0].imshow(np.asarray(u[Z//3], dtype=np.float64), vmin = vmin, vmax = vmax, cmap = cmap)
    ax[1].imshow(np.asarray(u[Z//2], dtype=np.float64), vmin = vmin, vmax = vmax, cmap = cmap)
    ax[2].imshow(np.asarray(u[2 * Z//3], dtype=np.float64), vmin = vmin, vmax = vmax, cmap = cmap)
    fig.tight_layout()
    plt.show()

# Convert nii.gz object <-> 3D np.array

import nibabel as nib

def save_nii(vol, filename, compressed = True) : 
    'Saves a 3D np.array into a .nii.gz or .nii file'
    img = nib.Nifti1Image(vol, np.eye(4))
    if compressed :
        if filename.endswith('.nii.gz') :
            filename = filename[:-7]
        img.to_filename(filename + '.nii.gz')
    else :
        if filename.endswith('.nii') :
            filename = filename[:-4]
        img.to_filename(filename + '.nii')

def load_nii(nii_file) :
    'Loads a .nii.gz or .nii file into a np.array'
    img = nib.load(nii_file)
    vol = np.asarray(img.get_data())
    return vol


## Computing SDPH

Using scikit-fmm (skfmm) to compute the signed distance

https://github.com/scikit-fmm/scikit-fmm

and giotto-tda (gtda) to compute persistent homology (PH)

https://github.com/giotto-ai/giotto-tda

In [ ]:
import skfmm
from gtda.homology import CubicalPersistence

def compute_PH(u) :
    distmap = -skfmm.distance(u >= 0, dx = 1) + skfmm.distance(u < 0, dx = 1)
    # IMPORTANT TO HAVE u >= 0 and u < 0 (convention : if u = 0, then assign strictly negative distance value)
    print(np.min(distmap), np.max(distmap))
    slices(distmap)

    CP = CubicalPersistence(homology_dimensions=[0, 1, 2], reduced_homology = False) 
    # NOT reduced homology to keep the 0th component living for infinity
    
    T0 = time.time()
    diagrams = CP.fit_transform(distmap[None])[0] # important to have distmap[None]!! and take [0] only
    T1 = time.time()
    
    print(diagrams.shape)
    print('Cubical Persistence computed in {} s '.format(T1 - T0)) 
    
    return diagrams
    

### Optional: close shape before SDPH

This step is not necessary in practice as it does not change SDPH diagrams a lot (but it does change them a little bit). Nevertheless, before computing SDPH, we can close a shape so that it does not meet the boundaries of the simulation domain.

In [ ]:
from skimage.filters import gaussian as sk_gaussian

def close_method_phi(u, N, dx = 5) :
    
    # CLOSING: method phi

    phi = np.ones((N,N,N))
    phi = np.pad(phi, dx, constant_values = 0)
    phi = sk_gaussian(phi, sigma=1, mode='nearest', cval= 0, preserve_range=False, truncate=4.0)

    v = np.pad(u, dx, mode = 'edge')
    w = phi * v + (1-phi) * (u.min() * np.ones([N+2*dx]*3))

    if 0 :
        slices(v)
        slices(w)
        slices(v>0) # small trap: show v not u (because slices in different places)
        slices(w>0)
    
    return w

## Plotting

Auxiliary functions for plotting SDPH diagrams (scattered points colored by density), using the keops library. 

https://github.com/getkeops/keops


In [ ]:
import torch
dtype = torch.cuda.FloatTensor

from pykeops.torch import LazyTensor

def gaussian_kde_keops(X, Y, weights = None, sigma = 1) : 
    ''' 
    Estimate gaussian density of X (nb of points, D) evaluated on points Y.
    Using the keops library makes sense only if you have a high number of points in the persistence diagrams,
    otherwise use any other gaussian KDE method (e.g. from sklearn) to estimate the same.
    There is a compiling time for keops the first time you define the symbolic formula.
    '''
    
    M = X.shape[0]
    N = Y.shape[0]
    if len(X.shape) == 1 :
        D = 1
    else :
        D = X.shape[1]
    X = torch.tensor(X)
    Y = torch.tensor(Y)

    x_i = LazyTensor(X.view(M, 1, D))
    y_j = LazyTensor(Y.view(1, N, D))

    D_ij = ((x_i - y_j)**2).sum(dim=2) 
    K_ij = (- D_ij / (2 * sigma**2)).exp()
    
    if weights is not None :
        weights = torch.tensor(weights)
        w_i = LazyTensor(weights.view(-1,1,1))
        K_ij = w_i * K_ij 
        sum_w_i = weights.sum()
    else :
        sum_w_i = M
    
    b_j = K_ij.sum(dim=0).view(-1) # torch.FloatTensor
    
    divide_factor = 2*torch.tensor(np.pi)*sigma**2 * sum_w_i

    res = b_j.numpy()
    res = res / divide_factor.numpy()
    return res


##### (ALTERNATIVELY, BUT SLOWER FOR LARGE NB OF PTS)

if False : # change to True if necessary
    
    from sklearn.neighbors import KernelDensity

    def gaussian_kde_sklearn(X, Y, weights = None, sigma = 1) : 
        kde = KernelDensity(kernel='gaussian', bandwidth=sigma).fit(X, sample_weight = weights)
        log_dens = kde.score_samples(Y) # log-density!!!!!!!
        res = np.exp(log_dens)
        return res

#####


def heatmap_keops(X, ax, weights = None, style = 'points', sigma = 1, 
                  cross_inf = False, size = 1, nb_bins_per_side = 100,
                  xlim = None, ylim = None, **kwargs)   :
    'Plots a beautiful colored scatter plot of a 2D histogram'  
    # X shape (Npoints, 2)

    if style == 'points' :
        Y = X
        x,y = Y.T
        inf_x = x[0] ; inf_y = y[0] # keep it before sorting
        
        z = gaussian_kde_keops(X,Y, weights = weights, sigma = sigma)
        
        # Sort the points by density, so that the densest points are plotted last
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]
        ax.scatter( x, y, c=z, s = size, zorder = 2, **kwargs )
        ax.grid(True, which='both', alpha = 0.3)
        if cross_inf :
            # add cross on inf val in PH0
            ax.scatter(inf_x, inf_y, s = 50, marker = 'x', c = 'r')

    if style in ['image', 'contour'] :

        if xlim is None or ylim is None :
            xlim = [ X[:,0].min()-1, X[:,0].max() + 1]
            ylim = [ X[:,1].min()-1, X[:,1].max() + 1]
          
        EXTENT = [xlim[0], xlim[1], ylim[0], ylim[1]]#[-1,1 + pi/2, -1, 1+pi/2] for ph_nets angles...
        II,JJ = np.mgrid[ylim[1] : ylim[0] : nb_bins_per_side * 1j,
                         xlim[0] : xlim[1] : nb_bins_per_side * 1j]
        y = II.flatten()
        x = JJ.flatten()
        Y = np.vstack((x,y)).T
        Y = np.ascontiguousarray(Y)
        z = gaussian_kde_keops(X,Y, weights = weights, sigma = sigma)
        img = z.reshape(II.shape)
        ax.imshow(img, extent = EXTENT, **kwargs)

    if style == 'contour' :
        ax.contour(img, extent = EXTENT, levels = 4,
                   origin = 'upper', cmap = 'inferno_r')
    if ax is None :
        plt.show()
    
    if style != 'points' :
        return img
    else :
        return None
        
def plot_PH_heatmap_keops(diagrams, dim, ax, inf_val = None, style = 'points', weights = True, sigma = 1, **kwargs):

    ph0 = len(np.where(diagrams[:,2] == 0)[0])
    ph1 = len(np.where(diagrams[:,2] == 1)[0])
    ph2 = len(np.where(diagrams[:,2] == 2)[0])
    cross_inf = False # deal with inf val or not (only for PH0)
    if dim == 0 :
        diag = diagrams[:ph0, :2]
        cross_inf = True
        if inf_val is not None :
            # special operation to have infinite PH0 death at given value
            diag[0,1] = inf_val
    if dim == 1 :
        diag = diagrams[ph0:ph0+ph1, :2]
    if dim == 2 :
        diag = diagrams[ph0 + ph1:, :2]

    if weights :
        weights = diag[:,1] - diag[:,0] # persist
    else :
        weights = None
    img = heatmap_keops(diag, ax, weights = weights, style = style, 
                        sigma = sigma, cross_inf = cross_inf, **kwargs)
    return img

# Gaussian Random Fields (GRF)

Generate GRF using GSTools:
https://geostat-framework.readthedocs.io/projects/gstools/en/stable/

We will use a Gaussian kernel. Other choices of kernel are possible: Exponential, Matern, Integral, Stable, Rational (see GSTools documentation).

## Generate GRF shapes

In [ ]:
GRF_shapes_folder = directory + 'GRF_shapes/' # folder to save generated fields / shapes


In [ ]:
from gstools import SRF, Gaussian
import gstools

# simulation domain

DIM = 3
N = 100 ; x = y = z = range(N)

# paramters of GRF generation

VAR = 1 # will not change the zero level set (if same seed)
LEN = 8
ANIS = 1 #[.7,.85] # either use ANIS = 1, or use 2 values in 3D (in first dim it will be ~ 1) specifying the ratios l2 / l1 and l3 / l1
ANG = 0 # use ANG = 0 or ANG = a, b, c (3 vals for 3D)

# optional: seed

SEED = 12345678

# Generate

model = Gaussian(dim=DIM, var=VAR, len_scale=LEN, anis=ANIS, angles=ANG)
srf = SRF(model, seed = SEED)

if DIM == 2 :    
    srf((x, y), mesh_type='structured')
if DIM == 3 :
    srf((x, y, z), mesh_type='structured')

u = srf.field
print(u.min(), u.max())


### Optional: close shape


In [ ]:
w = close_method_phi(u, N)

# Plot

if DIM == 2 : plt.figure(figsize = (3,3)) ; plt.imshow(w,cmap = 'jet') ; plt.show()
if DIM == 3 : slices(w)

In [ ]:
# Save generated GRF field

if ANIS == 1 :
    name = 'GRF_gau_var{}_len{}_iso_phi'.format(VAR, LEN)
else :
    # if ANG has three vals and you want to save them, use:
    #name = 'GRF_{}_var{}_len{}_anis{}_ang[{:.2f},{:.2f},{:.2f}]_phi'.format('gau', VAR, LEN, ANIS, *ANG)
    name = 'GRF_gau_var{}_len{}_anis{}_phi'.format(VAR, LEN, ANIS)
print(name)

if ERASE_AND_SAVE_YOURS :
    save_nii(np.floor(100 * w) / 100, GRF_shapes_folder + name, compressed = True)


### Field - value

We can modify a GRF field by changing all its values by a constant. This changes the positioning of the zero level set, and we may obtain more / less holes.

In [ ]:
# study u - du instead of u

du = .5

u = load_nii(GRF_shapes_folder + 'GRF_gau_var1_len5_iso_phi.nii.gz')
slices(u)

plt.imshow(u[55], cmap = 'jet')
plt.contour(u[55], levels = [0])
plt.show()

plt.imshow(u[55] - du, cmap = 'jet')
plt.contour(u[55] - du, levels = [0])
plt.show()

# save

if ERASE_AND_SAVE_YOURS :
    save_nii(u - du, GRF_shapes_folder + 'GRF_gau_var1_len5_iso_phi_decr{}.nii.gz'.format(du), compressed = True)

### Field1 + field2

In [ ]:
# u + v

u = load_nii(GRF_shapes_folder + 'GRF_gau_var1_len5_iso_phi.nii.gz')
v = load_nii(GRF_shapes_folder + 'GRF_gau_var1_len8_iso_phi.nii.gz')
slices(u)
slices(v)

img = (u+v)[55]
plt.imshow(img,cmap = 'jet')
plt.show()

# save

slices(u+v)

if ERASE_AND_SAVE_YOURS :
    save_nii(u + v, GRF_shapes_folder + 'GRF_gau_sum_len5and8.nii.gz', compressed = True)


## Compute SDPH

In [ ]:
shape_names = [nii_file[:-7] for nii_file in os.listdir(GRF_shapes_folder) if nii_file.endswith('.gz')]
print(shape_names)

GRF_shapes_PH_folder = directory + 'GRF_shapes_PH/'

In [ ]:
# compute SDPH for all shapes

for i in range(len(shape_names)) :
    print(shape_names[i])        

    PH_filepath = GRF_shapes_PH_folder + shape_names[i] + '.npy'

    if not os.path.exists(PH_filepath) :
        u = load_nii(GRF_shapes_folder + shape_names[i] + '.nii.gz')
        slices(u)

        diagrams = compute_PH(u)

        print(PH_filepath)
        
        np.save(PH_filepath, diagrams)
    else : print('existed')

# Curvatubes

Curvatubes is a random porous shape generator:
https://github.com/annasongmaths/curvatubes

Corresponding paper:
https://doi.org/10.1007/s10851-021-01049-9


## Generate cvtub shapes

To use the cvtub package, you need to download from the git and put the cvtub folder `curvatubes/cvtub/` at the same level as this notebook in your working directory.

Below is a snapshot of code showing how we can generate random porous shapes using curvatubes. To activate it, convert the `Raw` cells to `Code` cells.

We will not generate new shapes here, but instead directly use pre-generated shapes available on github at:
https://github.com/annasongmaths/curvatubes/tree/master/results/gallery
You may jump to the `Compute SDPH` subsection directly.

### Optional: close shapes

A piece of code showing how to close the shapes using the phi method, with the gallery of shapes provided at
https://github.com/annasongmaths/curvatubes/tree/master/results/gallery

## Compute SDPH

In [ ]:
cvtub_shapes_folder = directory + 'cvtub_shapes/'

shape_names = [nii_file[:-7] for nii_file in os.listdir(cvtub_shapes_folder) if nii_file.endswith('.nii.gz')]
print(shape_names)

cvtub_shapes_PH_folder = 'cvtub_shapes_PH/'

In [ ]:
# compute SDPH for all shapes

for i in range(len(shape_names)) :
    print(shape_names[i])
    PH_filepath = cvtub_shapes_PH_folder + shape_names[i] + '.npy'
    if not os.path.exists(PH_filepath) :
        u = load_nii(cvtub_shapes_folder + shape_names[i] + '.nii.gz')
        slices(u)
        diagrams = compute_PH(u)
        print(PH_filepath)
        np.save(PH_filepath, diagrams)
    else :
        print('existed')

# Bone marrow vessels

Three vascular shapes were segmented on 3D confocal images of bone marrow vessels.

After segmentation, denoising, and closing, the shapes are available in the `'BM_shapes/'` directory under the names:
- 'select_26_knee_crop_3_den_phi',
- 'select_26_long_crop_0_den_phi',
- 'select_26_long_crop_7_den_phi'


## Compute SDPH

In [ ]:
BM_shapes_folder = directory + 'BM_shapes/'

shape_names = [nii_file[:-7] for nii_file in os.listdir(BM_shapes_folder) if nii_file.endswith('.gz')
              if '_den_phi' in nii_file]
print(shape_names)

BM_shapes_PH_folder = directory + 'BM_shapes_PH/'

In [ ]:
# compute SDPH for all BM shapes

for i in range(len(shape_names)) :
    print(shape_names[i])        

    PH_filepath = BM_shapes_PH_folder + shape_names[i] + '.npy'

    if not os.path.exists(PH_filepath) :
        u = load_nii(BM_shapes_folder + shape_names[i] + '.nii.gz')
        slices(u)

        diagrams = compute_PH(u)

        print(PH_filepath)
        np.save(PH_filepath, diagrams)
    else : print('existed')

# SDPH: Texture and Stability

Experiments to show that SDPH diagrams of shapes generated with the same curvatubes parameters have similar SDPH diagrams. This suggests that SDPH diagrams do capture **texture** in shapes, and discards useless information related to the random geometric realization of the texture.

Furthermore, by changing generation parameters slightly, the SDPH diagrams only change slightly as well. This suggests some **stability behavior** of the SDPH diagrams w.r.t. texture (and not only geometric differences in terms of e.g. infinity-norm of the input functions f and g).


In [ ]:
stabfolder = 'stab_shapes/'

def convert_params(h2,H0,k1,s,kap1_0,t,kap2_0) :
    # h2 (H-H0)^2 + k1 K + s (kap1 - kap1_0)^2 + t (kap2 - kap2_0)^ 2
    a20 = h2 + s ; a11 = 2 * h2 + k1 ; a02 = h2 + t
    b10 = - 2*h2*H0 - 2*s*kap1_0 ; b01 = - 2*h2*H0 - 2*t*kap2_0 
    c = h2*H0**2 + s*kap1_0**2 + t * kap2_0**2
    coeffs = a20,a11,a02,b10,b01,c
    print(coeffs)
    return coeffs

In [ ]:
import torch
dtype = torch.cuda.FloatTensor

# with masking + spatial trick

Z,X,Y = 150,150,150
eps = 0.02
delta_x = 1/100
MAXEVAL = 8000

# fixed parameters
eps_s = torch.tensor(eps) # trick to make curvatubes know it's spatialized

## Generate shapes with same texture
Generate several shapes with same generation parameters in curvatubes.

In [ ]:
# LAUNCH 5 SHAPE GENERATIONS WITH SAME TEXTURE

# choose cvtub parameters

if True :
    # in "beta form" :
    h2,H0,k1,s,kap1_0,t,kap2_0 = 1, 15, .5, 5, 20, 5, -5
    a20, a11, a02, b10, b01, c = convert_params(h2,H0,k1,s,kap1_0,t,kap2_0) 
else :
    # in "alpha form" :
    a20, a11, a02, b10, b01, c = .....

# mass parameter
m = -0.5

for i in range(5) :

    params = eps_s, a20, a11, a02, b10, b01, c #, mu, theta 
    # mu, theta can be added if you want some orientation constraint

    A0 = 40 * delta_x * np.random.rand(3,Z,X,Y)
    A0 = torch.tensor(A0).type(dtype)

    u = generate(A0, params, m, delta_x, maxeval = MAXEVAL, snapshot_folder = niifolder)

    str_tup = tuple([ formatted(x) for x in [eps, a20, a11, a02, b10, b01, c, m, Z]])
    name = 'shape {} eps {} coeffs [{}, {}, {}, {}, {}, {}] m {} Z {}'.format(i, *str_tup)

    print(name)
    
    if ERASE_AND_SAVE_YOURS :
        save_nii(np.floor(100 * u) / 100, stabfolder + name, compressed = True)

    clear_output()

### Optional: close shapes

In [ ]:
# CLOSING: method phi

N = Z

for i in range(5) :
    name = 'shape {} eps {} coeffs [{}, {}, {}, {}, {}, {}] m {} Z {}'.format(i, *str_tup)
    u = load_nii(stabfolder + name + '.nii.gz')
    print('Closing ' + name)
    
    w = close_method_phi(u, N)

    print(stabfolder + name + '_phi')
    
    if ERASE_AND_SAVE_YOURS :
        save_nii(np.floor(100 * w) / 100, stabfolder + name + '_phi', compressed = True) #.nii or # .nii.gz ?


## Generate shapes with slightly different textures

In [ ]:
# LAUNCH 5 SHAPE GENERATIONS WITH SLIGHTLY DIFFERENT TEXTURES

# choose cvtub parameters

if True :
    # in "beta form" :
    h2,H0,k1,s,kap1_0,t,kap2_0 = 1, 15, .5, 5, 20, 5, -5
    a20, a11, a02, b10, b01, c = convert_params(h2,H0,k1,s,kap1_0,t,kap2_0) 
else :
    # in "alpha form" :
    a20, a11, a02, b10, b01, c = .....

# mass parameter
m = -0.5

for i in range(5) :

    # adding noise to the parameters
    a20_, a11_, a02_, b10_, b01_, c_ = np.array([a20, a11, a02, b10, b01, c]) + np.array([np.random.randn()*.01, np.random.randn()*.01, np.random.randn()*.01,
np.random.randn()*.1, np.random.randn()*.1, np.random.randn()*1])
    
    m_ = m + np.random.randn()*.05
    
    params_ = eps_s, a20_, a11_, a02_, b10_, b01_, c_ #, mu, theta 
    # mu, theta can be added if you want some orientation constraint

    A0 = 40 * delta_x * np.random.rand(3,Z,X,Y)
    A0 = torch.tensor(A0).type(dtype)

    u = generate(A0, params_, m_, delta_x, maxeval = MAXEVAL, snapshot_folder = niifolder)

    str_tup = tuple([ formatted(x) for x in [eps, a20, a11, a02, b10, b01, c, m, Z]])
    name = 'perturbed {} eps {} coeffs [{}, {}, {}, {}, {}, {}] m {} Z {}'.format(i, *str_tup)

    print(name)
    
    if ERASE_AND_SAVE_YOURS :
        save_nii(np.floor(100 * u) / 100, stabfolder + name, compressed = True)

    clear_output()

### Optional: close shapes

In [ ]:
# CLOSING: method phi

N = Z

for i in range(5) :
    name = 'perturbed {} eps {} coeffs [{}, {}, {}, {}, {}, {}] m {} Z {}'.format(i, *str_tup)
    u = load_nii(stabfolder + name + '.nii.gz')
    print('Closing ' + name)
    
    w = close_method_phi(u, N)

    print(stabfolder + name + '_phi')
    
    if ERASE_AND_SAVE_YOURS :
        save_nii(np.floor(100 * w) / 100, stabfolder + name + '_phi', compressed = True)


## Compute SDPH

In [ ]:
stab_shapes_folder = directory + 'stab_shapes/'

shape_names = [nii_file[:-7] for nii_file in os.listdir(stab_shapes_folder) if nii_file.endswith('.gz')
              if '_phi' in nii_file]
print(shape_names)

stab_shapes_PH_folder = directory + 'stab_shapes_PH/'

In [ ]:
# compute SDPH for all shapes

for i in range(len(shape_names)) :
    print(shape_names[i])        

    PH_filepath = stab_shapes_PH_folder + shape_names[i] + '.npy'

    if not os.path.exists(PH_filepath) :
        u = load_nii(stab_shapes_folder + shape_names[i] + '.nii.gz')
        slices(u)

        diagrams = compute_PH(u)

        print(PH_filepath)
        np.save(PH_filepath, diagrams)
    else : print('existed')

# Plot SDPH diagrams

In [ ]:
## Choose plotting parameters

# kernel size in density estimation
SIGMA = .5

# if True, add a weight to each (birth, death) point, equal to the persistence death - birth
# else False, all points have equal unit weight
WEIGHTS = False

# threshold out low-persistence points (with death - birth < .5)
THR = .5

# determine style of plots
if True :
    style = 'points'
    img_cmap = 'magma'
else :
    style = 'contour'
    img_cmap = 'Oranges'


In [ ]:
## Choose which panel 

choice = 'cvtub' # 'BM', 'GRF', 'stab'

shapes_folder = directory + '{}_shapes/'.format(choice)
shapes_PH_folder = directory = '{}_shapes_PH/'.format(choice)

shape_names = [PH_file[:-4] for PH_file in os.listdir(shapes_PH_folder) if PH_file.endswith('.npy')]
print(shape_names)

In [ ]:
if choice == 'stab' :
    shape_names = sorted(shape_names)

    xlims = [[-8,5],[-5,15],[-5,15]]
    ylims = [[-5,8],[-5,15],[-5,15]]
    idlims = [[None,None],[-5,15],[-5,15]]

elif choice == 'BM' :
    shape_names = sorted(shape_names)
    
    xlims = [[-10,5],[-8,18],[-5,20]]
    ylims = [[-10,5],[-8,18],[-5,20]]
    idlims = [[-10,5],[-8,18],[-5,20]]
    
elif choice == 'GRF' :
    shape_names = sorted(shape_names)
    
    xlims = [[-10,5],[-10,15],[-10,15]]
    ylims = [[-10,5],[-10,15],[-10,15]]
    idlims = [[-10,5],[-10,15],[-10,15]]
    
elif choice == 'cvtub' :
    
    xlims = [[-8,5],[-5,15],[-5,15]]
    ylims = [[-5,8],[-5,15],[-5,15]]
    idlims = [[None,None],[-5,15],[-5,15]]


## One by one

In [ ]:
def plot_cool_PH(diagrams,SIGMA,WEIGHTS,inf_val = None, style = 'contour', img_cmap = 'magma') : 
    # 'Oranges' good for contour
    # 'magma' good for points

    fig, axes = plt.subplots(1,3,figsize = (8,3))#, sharex = True, sharey = True)
    for dim in range(3) :
        ax = axes[dim]
        plot_PH_heatmap_keops(diagrams, dim, ax, style = style, # 'points', 'image', 'contour'
                              inf_val=inf_val,weights = WEIGHTS, sigma = SIGMA, cmap = img_cmap,
                              xlim = xlims[dim],
                              ylim = ylims[dim])
        ax.axhline(y=0, color='k', linewidth = 1, alpha = 0.8)
        ax.axvline(x=0, color='k', linewidth = 1, alpha = 0.8)
        ax.plot([idlims[dim][0], idlims[dim][1]],[idlims[dim][0], idlims[dim][1]], 
                c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
        ax.set_aspect('equal')
    for dim in range(3) :
        axes[dim].set_xlabel('PH{}'.format(dim))

    plt.tight_layout()
    #if save_fig : plt.savefig(figure_path, dpi = 200) ; print('saved in', figure_path)
    plt.show()

In [ ]:
# load and plot PH


for i in range(len(shape_names)) :
    PH_filepath = shapes_PH_folder + shape_names[i] + '.npy'
    nii_filepath = shapes_folder + shape_names[i] + '.nii.gz'
    print(PH_filepath)
    diagrams = np.load(PH_filepath)
    diagrams = diagrams[diagrams[:,1] >= diagrams[:,0] + THR]

    u = load_nii(nii_filepath)
    
    death_vals_PH0 = diagrams[diagrams[:,2] == 0][1:,1] # real deaths
    max_death = max(death_vals_PH0)
    
    #plot_cool_PH(diagrams,SIGMA,WEIGHTS, style = 'contour', img_cmap = 'Oranges')
    plot_cool_PH(diagrams,SIGMA,WEIGHTS,inf_val=max(max_death+1, 10), style = 'points', img_cmap = 'magma')
    slices(u)

## Paper figs

In each panel, one row corresponds to one shape. From the leftmost column to the rightmost column, we plot: 
- a 3D view of the shape (before being closed). We use Paraview to have a nice picture;
- a 2D section of the shape (after closing);
- the 3 last columns show PH0, PH1, PH2.

### GRF

In [ ]:
#### GRF VERSION

# 3D view + slice + PH

gallery_folder = directory + 'GRF_shapes/'

# GAU: PAPER FIGS
list_names = ['GRF_gau_var1_len8_iso_phi',
              'GRF_gau_var1_len8_anis[0.7, 0.85]_ang[0.00,2.09,0.63]_phi',
              'GRF_gau_var1_len5_iso_phi',
              'GRF_gau_var1_len5_iso_phi_decr0.5',
              'GRF_gau_sum_len5and8'
              ]

M = len(list_names)
fig, axes = plt.subplots(M,5,figsize = (8+3+3,3*M))

for i in range(M) :
    PH_filepath = shapes_PH_folder + list_names[i] + '.npy'
    nii_filepath = shapes_folder + list_names[i] + '.nii.gz'
    print(PH_filepath)
    
    diagrams = np.load(PH_filepath)
    diagrams = diagrams[diagrams[:,1] >= diagrams[:,0] + THR]
    death_vals_PH0 = diagrams[diagrams[:,2] == 0][1:,1] # real deaths
    max_death = max(death_vals_PH0)
    inf_val=max(max_death+1, 5) # 10

    ax = axes[i,0]
    #ax.set_xlabel('{}'.format(list_names[i][:]),fontsize = 4)#[-7:]
    im = imageio.imread(gallery_folder + list_names[i] + '_nopad.png')
    I,J,C = im.shape
    ax.imshow(im[int(.05*I):, int(.24*J) : 3*J//4])
    ax.axis('off')
    
    ax = axes[i,1] #1
    u = load_nii(nii_filepath)
    im = ax.imshow(u[u.shape[0]//2],cmap = 'jet')
    ax.contour(u[u.shape[0]//2], levels = [0])
    ax.axis('off')
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('bottom', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='horizontal')
    
    for dim in range(3) :
        ax = axes[i, 2+dim] # 2
        plot_PH_heatmap_keops(diagrams, dim, ax, style = style,
                              inf_val=inf_val,weights = WEIGHTS, sigma = SIGMA, cmap = img_cmap,
                              xlim = xlims[dim],
                              ylim = ylims[dim])
        ax.axhline(y=0, color='k', linewidth = 1, alpha = 0.8)
        ax.axvline(x=0, color='k', linewidth = 1, alpha = 0.8)
        ax.plot([idlims[dim][0], idlims[dim][1]],[idlims[dim][0], idlims[dim][1]], 
                c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
        ax.set_aspect('equal')
    for dim in range(3) :
        axes[i, 2+dim].set_xlabel('PH{}'.format(dim)) # 1

#axes[-1,0].set_xlabel('3D view')
#axes[-1,1].set_xlabel('mid slice')

plt.tight_layout()

figure_path = shapes_PH_folder + 'PAPER GRF sig{}_wght{}_thr{}.png'.format(SIGMA,WEIGHTS,THR)
if not os.path.exists(figure_path) :
    plt.savefig(figure_path, dpi = 300) ; print('saved in', figure_path)
else :
    print('Figure already existed')
plt.show()
plt.close()

### Curvatubes

Do not forget to download the 3D views at https://github.com/annasongmaths/curvatubes/tree/master/results/gallery

In [ ]:
#### GIT CVTB VERSION

# 3D view + slice + PH

gallery_folder = directory + 'curvatubes/results/gallery/'

cvtub_panel_choice = 'CVTUB PANEL 1' # 'CVTUB PANEL 1', 'CVTUB PANEL 2'

if True : # panel 1
    list_names = ['tubes coeffs [1 2 6 -40 -40 400] m -0.6_phi',
                  'exp 0253 coeffs [1 -1.284 9.626 -77.283 39.681 -633.849] m -0.421_phi',
                  'sponge coeffs [1 2.8 2 -10 -10 25] m -0.25_phi',
                  'exp 0556 coeffs [1 3.225 10.87 -146.309 143.487 -2920.302] m -0.499_phi',
                  'exp 0366 coeffs [1 -0.238 11.988 -175.909 -27.167 2117.037] m -0.648_phi']
else : # panel 2
    list_names = ['exp 0716 coeffs [1 2.034 11.166 14.553 28.829 -565.092] m -0.356_phi',
                  'exp 0374 coeffs [1 0.63 4.399 132.459 195.066 -2378.53] m -0.364_phi',
                  'bilayers coeffs [1 1 1 0 0 0] m -0.4_phi',
                  'weird coeffs [1 0.396 1.095 -28.64 190.906 2062.082] m -0.598_phi',
                  'sieve coeffs [0.946 3.959 1.942 18.329 28.008 113.771] m -0.534_phi']

M = len(list_names)
fig, axes = plt.subplots(M,5,figsize = (8+3+3,3*M))

for i in range(M) :
    PH_filepath = shapes_PH_folder + list_names[i] + '.npy'
    nii_filepath = shapes_folder + list_names[i] + '.nii.gz'
    print(PH_filepath)
    
    diagrams = np.load(PH_filepath)
    diagrams = diagrams[diagrams[:,1] >= diagrams[:,0] + THR]
    death_vals_PH0 = diagrams[diagrams[:,2] == 0][1:,1] # real deaths
    max_death = max(death_vals_PH0)
    inf_val=max(max_death+1, 8) # 10

    ax = axes[i,0]
    im = imageio.imread(gallery_folder + list_names[i][:-4] + '.png') # discard _phi
    I,J,C = im.shape
    ax.imshow(im[:, int(.22*J) : 3*J//4]) # between 1/5 and 1/4
    ax.axis('off')
    
    ax = axes[i,1]
    u = load_nii(nii_filepath)
    ax.imshow(u[u.shape[0]//2],cmap = 'gray')
    ax.axis('off')
    
    for dim in range(3) :
        ax = axes[i, 2+dim]
        plot_PH_heatmap_keops(diagrams, dim, ax, style = style,
                              inf_val=inf_val,weights = WEIGHTS, sigma = SIGMA, cmap = img_cmap,
                              xlim = xlims[dim],
                              ylim = ylims[dim])
        ax.axhline(y=0, color='k', linewidth = 1, alpha = 0.8)
        ax.axvline(x=0, color='k', linewidth = 1, alpha = 0.8)
        if dim in [1,2] :
            ax.plot([idlims[dim][0], idlims[dim][1]],[idlims[dim][0], idlims[dim][1]], 
                    c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
        if dim == 0 :
            ax.plot([-5,5],[-5,5], 
                c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
            
            # ASTUCE DE LA MORT pour decaler frame vers gauche bas
            ax.scatter([-8],[-5],c = 'w')
        
        ax.set_aspect('equal')
    for dim in range(3) :
        axes[i, 2+dim].set_xlabel('PH{}'.format(dim))

#axes[-1,0].set_xlabel('3D view')
#axes[-1,1].set_xlabel('mid slice')

plt.tight_layout()
figure_path = shapes_PH_folder + 'PAPER {} sig{}_wght{}_thr{}.png'.format(cvtub_panel_choice, SIGMA,WEIGHTS,THR)

if not os.path.exists(figure_path) :
    plt.savefig(figure_path, dpi = 300) ; print('saved in', figure_path)
else :
    print('Figure already existed')
plt.show()
plt.close()

### Stability

In [ ]:
# 3D view + slice + PH

gallery_folder = directory + 'stab_shapes/'

# Z = 150 ITER = 8000 panel
list_names = ['shape 0 eps 0.02 coeffs [6, 2.5, 6, -230, 20, 2350] m -0.5 Z 150_phi', 
              'shape 1 eps 0.02 coeffs [6, 2.5, 6, -230, 20, 2350] m -0.5 Z 150_phi',
              'shape 2 eps 0.02 coeffs [6, 2.5, 6, -230, 20, 2350] m -0.5 Z 150_phi',
              'shape 3 eps 0.02 coeffs [6, 2.5, 6, -230, 20, 2350] m -0.5 Z 150_phi',
              'shape 4 eps 0.02 coeffs [6, 2.5, 6, -230, 20, 2350] m -0.5 Z 150_phi']

M = len(list_names)
fig, axes = plt.subplots(M,5,figsize = (8+3+3,3*M))

for i in range(M) :
    PH_filepath = shapes_PH_folder + list_names[i] + '.npy'
    nii_filepath = shapes_folder + list_names[i] + '.nii.gz'
    print(PH_filepath)
    
    diagrams = np.load(PH_filepath)
    diagrams = diagrams[diagrams[:,1] >= diagrams[:,0] + THR]
    death_vals_PH0 = diagrams[diagrams[:,2] == 0][1:,1] # real deaths
    max_death = max(death_vals_PH0)
    inf_val=max(max_death+1, 8) # 10

    ax = axes[i,0]
    im = imageio.imread(gallery_folder + list_names[i][:-4] + '.png') # discard _phi
    I,J,C = im.shape
    #ax.imshow(im[:, int(.22*J) : 3*J//4])
    ax.imshow(im[int(.02*I):int(.98*I), int(.22*J) : int(.73*J)]) # Z 150

    ax.axis('off')
    
    ax = axes[i,1]
    u = load_nii(nii_filepath)
    ax.imshow(u[u.shape[0]//2],cmap = 'gray')
    ax.axis('off')
    #ax.set_xlabel('ind = {}'.format(selection[i]))
    
    for dim in range(3) :
        ax = axes[i, 2+dim]
        plot_PH_heatmap_keops(diagrams, dim, ax, style = style,
                              inf_val=inf_val,weights = WEIGHTS, sigma = SIGMA, cmap = img_cmap,
                              xlim = xlims[dim],
                              ylim = ylims[dim])
        ax.axhline(y=0, color='k', linewidth = 1, alpha = 0.8)
        ax.axvline(x=0, color='k', linewidth = 1, alpha = 0.8)
        if dim in [1,2] :
            ax.plot([idlims[dim][0], idlims[dim][1]],[idlims[dim][0], idlims[dim][1]], 
                    c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
        if dim == 0 :
            ax.plot([-5,5],[-5,5], 
                c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
            
            # ASTUCE DE LA MORT pour decaler frame vers gauche bas
            ax.scatter([-8],[-5],c = 'w')
        
        ax.set_aspect('equal')
    for dim in range(3) :
        axes[i, 2+dim].set_xlabel('PH{}'.format(dim))

plt.tight_layout()
figure_path = shapes_PH_folder + 'PAPER SAMETEXT sig{}_wght{}_thr{}.png'.format(SIGMA,WEIGHTS,THR)

if not os.path.exists(figure_path) :
    plt.savefig(figure_path, dpi = 300) ; print('saved in', figure_path)
else :
    print('Figure already existed')
plt.show()
plt.close()

### BM vessels

In [ ]:
#### BONE MARROW VESSELS

# 3D view + slice + PH

gallery_folder = directory + 'BM_shapes/'

list_names = ['select_26_knee_crop_3_den_phi',
              'select_26_long_crop_0_den_phi',
              'select_26_long_crop_7_den_phi']
    
M = len(list_names)
fig, axes = plt.subplots(M,5,figsize = (8+3+3,3*M))

for i in range(M) :
    PH_filepath = shapes_PH_folder + list_names[i] + '.npy'
    nii_filepath = shapes_folder + list_names[i] + '.nii.gz'
    print(PH_filepath)
    
    diagrams = np.load(PH_filepath)
    diagrams = diagrams[diagrams[:,1] >= diagrams[:,0] + THR]
    death_vals_PH0 = diagrams[diagrams[:,2] == 0][1:,1] # real deaths
    max_death = max(death_vals_PH0)
    inf_val=max(max_death+1, 5) # 10

    ax = axes[i,0]
    im = imageio.imread(gallery_folder + list_names[i][:-4] + '.png') # discard _phi
    I,J,C = im.shape
    ax.imshow(im[50:, int(.24*J) : 3*J//4])
    ax.axis('off')
    
    ax = axes[i,1]
    u = load_nii(nii_filepath)
    ax.imshow(u[u.shape[0]//2],cmap = 'gray')
    ax.axis('off')
    #ax.set_xlabel('ind = {}'.format(selection[i]))
    
    for dim in range(3) :
        ax = axes[i, 2+dim]
        plot_PH_heatmap_keops(diagrams, dim, ax, style = style,
                              inf_val=inf_val,weights = WEIGHTS, sigma = SIGMA, cmap = img_cmap,
                              xlim = xlims[dim],
                              ylim = ylims[dim])
        ax.axhline(y=0, color='k', linewidth = 1, alpha = 0.8)
        ax.axvline(x=0, color='k', linewidth = 1, alpha = 0.8)
        if dim in [1,2] :
            ax.plot([idlims[dim][0], idlims[dim][1]],[idlims[dim][0], idlims[dim][1]], 
                    c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
        if dim == 0 :
            ax.plot([-11,5],[-11,5], 
                c = 'k',alpha = .2, linewidth = 1, label='_nolegend_')
        
        ax.set_aspect('equal')
    for dim in range(3) :
        axes[i, 2+dim].set_xlabel('PH{}'.format(dim))

#axes[-1,0].set_xlabel('3D view')
#axes[-1,1].set_xlabel('mid slice')

plt.tight_layout()
figure_path = shapes_PH_folder + 'PAPER BM sig{}_wght{}_thr{}.png'.format(SIGMA,WEIGHTS,THR)

if not os.path.exists(figure_path) :
    plt.savefig(figure_path, dpi = 300) ; print('saved in', figure_path)
else :
    print('Figure already existed')
plt.show()
plt.close()